In [ ]:
%matplotlib inline
from IPython.display import IFrame

### usual imports

In [ ]:
import os
import sys
from glob import glob 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

### fbprophet 

[fbprophet](https://facebook.github.io/prophet/) implement a [Generalized Additive Model](https://en.wikipedia.org/wiki/Generalized_additive_model) in Python and R, and is extremely convenient for the modelling of time-series which contain a **trend** (potentially non-linear), some **cyclic components** (seasonal cycle, weekly cycle, daily cycle) and potentially respond
to **`pulse` events** which do not obey a regular schedule, the latter is particularly important for e.g. time-series of sales data, where holidays, special events, marketing campains, etc, might drive short term, irregular increases or decrease in sales volume / value.   

And of course, of interest for us is the fact that you can easily add **extra regressors** to the model: in our case it is likely to be climate / weather variables, and the formulation of the model makes is relatively easy to quantify the added value derived from incorporating climate / weather explanatory variables to the model.

to know more about the **Generalized Additive Model** framework, a good start is:   
    
[https://multithreaded.stitchfix.com/assets/files/gam.pdf](https://multithreaded.stitchfix.com/assets/files/gam.pdf)

In [ ]:
IFrame(src='https://facebook.github.io/prophet/', width=1500, height=600)

In [ ]:
from fbprophet import Prophet

### scikit learn Mean Absolute Error metrics function (MAE)

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

### seaborn for visualisation 

In [ ]:
import seaborn as sns

### read the data: cycling counts over Tamaki drive 

the data is taken from [https://awcc.mrcagney.works/](https://awcc.mrcagney.works/)

In [ ]:
from IPython.display import IFrame

In [ ]:
IFrame(src='https://awcc.mrcagney.works/', width=1500, height=600)

### for a better view of the location of the counters, we're gonna create an interactive map in the Jupyter Notebook using [https://github.com/python-visualization/folium](https://github.com/python-visualization/folium)

#### reads in the locations of the counters 

In [ ]:
loc_counters = pd.read_csv('../data/cycling_Auckland/cycling_counters.csv')

In [ ]:
loc_counters.head()

#### we're only keeping the counters for cyclists

In [ ]:
loc_counters = loc_counters.query("user_type == 'Cyclists'")

### we're gonna centre the map on the Tamaki Drive counter 

In [ ]:
loc_counters.query("name == 'Tamaki Drive EB'")

In [ ]:
center_lat = loc_counters.query("name == 'Tamaki Drive EB'").latitude.values[0]
center_lon = loc_counters.query("name == 'Tamaki Drive EB'").longitude.values[0]

### display the counters locations on an interactive map

In [ ]:
import folium
from folium.plugins import MarkerCluster

In [ ]:
folium.__version__

this is the development version of folium

In [ ]:
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=13,
    tiles='OpenStreetMap', 
    width='80%', 
)

m.add_child(folium.LatLngPopup())

# marker_cluster = MarkerCluster().add_to(m)

# loops over each row in the DataFrame holding the counters information, and 
# adds to the map 

for i, row in loc_counters.iterrows():
    name = row['name']
    lat = row.latitude
    lon = row.longitude
    opened = row.setup_date
    
    # HTML here in the pop up 
    popup = '<b>{}</b></br><i>setup date = {}</i>'.format(name, opened)
    
#     folium.Marker([lat, lon], popup='<i>{}</i>'.format(name), tooltip=name).add_to(marker_cluster)
    folium.Marker([lat, lon], popup=popup, tooltip=name).add_to(m)

### now display the map 

In [ ]:
m

### now reads the cycling counts

#### get the list of files (one per year)

In [ ]:
lfiles = glob('../data/cycling_Auckland/cycling_counts_????.csv')

#### sort

In [ ]:
lfiles.sort()

In [ ]:
lfiles

#### loops over the list of files, reads and keep the Tamaki Drive data, and concatenate 

In [ ]:
l = []
for f in lfiles: 
    d = pd.read_csv(f, index_col=0, parse_dates=True, usecols=['datetime', 'Tamaki Drive EB','Tamaki Drive WB'])
    l.append(d)

In [ ]:
df = pd.concat(l, axis=0)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.plot(subplots=True, figsize=(10, 8));

### seasonal cycle (30 days running window for smoothing) 

In [ ]:
f, ax = plt.subplots(figsize=(8,6)) 
df.rolling(window=30*24, center=True).sum().groupby(df.index.dayofyear).mean().plot(ax=ax)
ax.grid(ls=':')
ax.set_xlabel('day of year')

### weekly cycle 

In [ ]:
f, ax = plt.subplots(figsize=(8,6)) 
df.groupby(df.index.dayofweek).mean().plot(ax=ax)
ax.grid(ls=':')
ax.set_xlabel('day of week')

### daily cycle 

In [ ]:
summary_hour = df.groupby(df.index.hour).describe()

In [ ]:
summary_hour.head()

In [ ]:
f, ax = plt.subplots(figsize=(8,7))

ax.plot(summary_hour.index, summary_hour.loc[:,('Tamaki Drive EB','mean')], color='b', label='Eastern Bound')
ax.plot(summary_hour.index, summary_hour.loc[:,('Tamaki Drive WB','mean')], color='r', label='Western Bound')

ax.fill_between(summary_hour.index, summary_hour.loc[:,('Tamaki Drive WB','25%')], \
                summary_hour.loc[:,('Tamaki Drive WB','75%')], color='coral', alpha=0.3)

ax.fill_between(summary_hour.index, summary_hour.loc[:,('Tamaki Drive EB','25%')], \
                summary_hour.loc[:,('Tamaki Drive EB','75%')], color='steelblue', alpha=0.3)

ax.legend(loc=2)

ax.set_xticks(range(24));
ax.grid(ls=':', color='0.8')

ax.set_xlabel('hour of the day')

ax.set_ylabel('cyclists number');

### getting rid of the outliers using a median filter 

In [ ]:
df.plot(subplots=True, figsize=(10, 8), **{'ylim':[0, 600]});

In [ ]:
def median_filter(df, varname = 'Tamaki Drive EB', window=24, std=2): 
    
    dfc = df.copy() 
    
    dfc = dfc.loc[:,[varname]]
    
    dfc['median']= dfc[varname].rolling(window, center=True).median()
    
    dfc['std'] = dfc[varname].rolling(window, center=True).std()
    
    dfc.loc[dfc.loc[:,varname] >= dfc['median']+std*dfc['std'], varname] = np.nan
    
    dfc.loc[dfc.loc[:,varname] <= dfc['median']-std*dfc['std'], varname] = np.nan
    
    return dfc.loc[:, varname]

In [ ]:
df_filtered = df.copy()

In [ ]:
window = 24

In [ ]:
std = 2

In [ ]:
varname = 'Tamaki Drive EB'

In [ ]:
df_filtered.loc[:,varname] = median_filter(df_filtered, varname = varname, window=window, std=std)

In [ ]:
varname = 'Tamaki Drive WB'

In [ ]:
df_filtered.loc[:,varname] = median_filter(df_filtered, varname = varname, window=window, std=std)

In [ ]:
df_filtered.plot(subplots=True, figsize=(10, 8), **{'ylim':[0, 600]});

### resampling at the daily time-step


In [ ]:
df_filtered = df_filtered.resample('1D').sum()

In [ ]:
df_filtered.head()

In [ ]:
df_filtered.tail()

### we're gonna look at the cyclists count for Eastern Bound Tamaki Drive ... starting in 2011 as it is when the climate data starts

In [ ]:
data = df_filtered.loc['2011':,['Tamaki Drive EB']]

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
data.plot(ax=ax)
ax.grid(ls=':')
f.savefig('../figures/cycling_counts_Tamaki_drive_EB.png', dpi=200)

### preparing the data 

In [ ]:
data = data.rename({'Tamaki Drive EB':'y'}, axis=1)

In [ ]:
data.tail()

### defining here a few utility functions to do all the 'data wrangling'

In [ ]:
def add_regressor(data, regressor, varname=None): 
    
    """
    adds a regressor to a dataframe of targets
    """
    
    data.loc[:,varname] = regressor.loc[:,varname]
    
    return data

In [ ]:
def prepare_data(data, year=2017): 
    
    """
    prepare the data for ingestion by fbprophet: 
    
    1) divide in training and test set, using the `year` parameter (int)
    
    2) reset the index and rename the `datetime` column to `ds`
    
    returns the training and test dataframes
    """
    
    
    data_train = data.loc[:str(year - 1),:]
    
    data_test = data.loc[str(year):,:]
    
    data_train.reset_index(inplace=True)
    
    data_test.reset_index(inplace=True)
    
    data_train = data_train.rename({'datetime':'ds'}, axis=1)
    
    data_test = data_test.rename({'datetime':'ds'}, axis=1)
    
    return data_train, data_test

In [ ]:
def make_verif(forecast, data_train, data_test): 
    """
    put together the forecast (coming from fbprophet) 
    and the overved data, and set the index to be a proper datetime index, 
    for plotting
    
    """
    
    forecast.index = pd.to_datetime(forecast.ds)
    
    data_train.index = pd.to_datetime(data_train.ds)
    
    data_test.index = pd.to_datetime(data_test.ds)
    
    data = pd.concat([data_train, data_test], axis=0)
    
    forecast.loc[:,'y'] = data.loc[:,'y']
    
    return forecast

In [ ]:
def plot_verif(verif, year=2017):
    """
    plots the forecasts and observed data, the year parameters is used to highlight 
    the difference between the training and test data 
    """
    
    f, ax = plt.subplots(figsize=(10, 8))
    
    train = verif.loc[:str(year - 1),:]
    
    ax.plot(train.index, train.y, 'ko', markersize=3)
    
    ax.plot(train.index, train.yhat, color='steelblue', lw=0.5)
    
    ax.fill_between(train.index, train.yhat_lower, train.yhat_upper, color='steelblue', alpha=0.3)
    
    test = verif.loc[str(year):,:]
    
    ax.plot(test.index, test.y, 'ro', markersize=3)
    
    ax.plot(test.index, test.yhat, color='coral', lw=0.5)
    
    ax.fill_between(test.index, test.yhat_lower, test.yhat_upper, color='coral', alpha=0.3)
    
    ax.axvline(str(year), color='0.8', alpha=0.7)
    
    ax.grid(ls=':', lw=0.5)
    
    return f

In [ ]:
def plot_verif_component(verif, component='rain', year=2017): 
    """
    plots a specific component of the model
    """
    
    f, ax = plt.subplots(figsize=(10, 8))
    
    train = verif.loc[:str(year - 1),:]
        
    ax.plot(train.index, train.loc[:,component], color='steelblue', lw=0.5)
    
    ax.fill_between(train.index, train.loc[:, component+'_lower'], train.loc[:, component+'_upper'], color='steelblue', alpha=0.3)
    
    test = verif.loc[str(year):,:]
        
    ax.plot(test.index, test.loc[:,component], color='coral', lw=0.5)
    
    ax.fill_between(test.index, test.loc[:, component+'_lower'], test.loc[:, component+'_upper'], color='coral', alpha=0.3)
    
    ax.axvline(str(year), color='0.8', alpha=0.7)
    
    ax.grid(ls=':', lw=0.5)
    
    return f

In [ ]:
def add_regressor_to_future(future, regressors_list): 
    """
    adds extra regressors to a `future` dataframe created by fbprophet
    """
    
    futures = future.copy() 
    
    futures.index = pd.to_datetime(futures.ds)
    
    regressors = pd.concat(regressors_list, axis=1)
    
    futures = futures.merge(regressors, left_index=True, right_index=True)
    
    futures = futures.reset_index(drop = True) 
    
    return futures

### splits the data into a training and test set, and returns these data frames in a format **fbprophet** can understand 

In [ ]:
data_train, data_test = prepare_data(data, 2017)

In [ ]:
data_train.head()

In [ ]:
data_test.head()

### instantiates, then fit the model to the training data 

The first step in **fbprophet** is to instantiate the model, it is there that you can set the `prior scales` for each component of your time-series, as well as the number of Fourier series to use to model the cyclic components.   

A general rule is that larger prior scales and larger number of Fourier series will make the model more flexible, but at the potential cost of generalisation: i.e. the model might [overfit](https://en.wikipedia.org/wiki/Overfitting), learning the noise (rather than the signal) in the training data, but 
    giving poor results when applied to yet unseen data (the test data)... setting these [hyperparameters](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)) can be more an art than a science ... 

In [ ]:
m = Prophet(mcmc_samples=300, changepoint_prior_scale=0.01, seasonality_mode='multiplicative', \
            yearly_seasonality=10, \
            weekly_seasonality=True, \
            daily_seasonality=False)

In [ ]:
m.fit(data_train)

### make the `future` dataframe 

In [ ]:
future = m.make_future_dataframe(periods=len(data_test), freq='1D')

In [ ]:
future.head()

In [ ]:
future.tail()

In [ ]:
forecast = m.predict(future)

### plots the `components` of the forecast (trend + cyclic component [yearly seasonality, weekly seasonality] at this stage)

In [ ]:
f = m.plot_components(forecast)

### put it all together with the actual observations 

In [ ]:
verif = make_verif(forecast, data_train, data_test)

In [ ]:
f = plot_verif(verif)

### scatter plot, marginal distribution and correlation between observations and modelled / predicted values 

In [ ]:
sns.jointplot(x='yhat', y='y', data = verif.loc[:'2017',:]);

In [ ]:
sns.jointplot(x='yhat', y='y', data = verif.loc['2017':,:])

### Mean Absolute Error (in number of cyclists)

In [ ]:
MAE(verif.y.values, verif.yhat.values)

## now incorporating the effects of the holidays 

In [ ]:
holidays_calendar = pd.read_csv('../data/holidays_calendars_2011_2018.csv')

In [ ]:
holidays_calendar.loc[:,'ISO_date'] = pd.to_datetime(holidays_calendar.loc[:,'ISO_date'], dayfirst=True)

In [ ]:
holidays_calendar = holidays_calendar.dropna(axis=1, how='all').dropna(axis=0, how='all')

In [ ]:
holidays_calendar = holidays_calendar.loc[-holidays_calendar.notes.str.contains('Not a public'),:]

In [ ]:
holidays = holidays_calendar.loc[(holidays_calendar.loc[:,'Regional'] == 0) | holidays_calendar.RGR.str.contains('Auckland'),:]

In [ ]:
holtype = 'category'

In [ ]:
if holtype == 'category': 
    holidays = holidays.loc[:,['ISO_date','holiday_category']]
    holidays = holidays.rename({'holiday_category':'holiday'}, axis=1)
if holtype == 'code': 
    holidays = holidays.loc[:,['ISO_date','holiday_code']]
    holidays = holidays.rename({'holiday_code':'holiday'}, axis=1)
else: 
    holidays = holidays.loc[:,['ISO_date','holiday']]

In [ ]:
holidays = holidays.rename({'ISO_date':'ds'}, axis=1)

In [ ]:
holidays.tail()

In [ ]:
m = Prophet(mcmc_samples=300, holidays=holidays, holidays_prior_scale=0.25, changepoint_prior_scale=0.01, seasonality_mode='multiplicative', \
            yearly_seasonality=10, \
            weekly_seasonality=True, \
            daily_seasonality=False)

In [ ]:
m.fit(data_train)

In [ ]:
future = m.make_future_dataframe(periods=len(data_test), freq='1D')

In [ ]:
forecast = m.predict(future)

In [ ]:
f = m.plot_components(forecast)

In [ ]:
verif = make_verif(forecast, data_train, data_test)

In [ ]:
plot_verif(verif)

In [ ]:
sns.jointplot(x='yhat', y='y', data = verif.loc[:'2017',:])

In [ ]:
sns.jointplot(x='yhat', y='y', data = verif.loc['2017':,:])

In [ ]:
MAE(verif.y.values, verif.yhat.values)

## incorporating the effects of weather conditions

In [ ]:
temp = pd.read_csv('../data/weather/Mangere_EWS_temp.csv', index_col=0, parse_dates=True)

In [ ]:
rain = pd.read_csv('../data/weather/Mangere_EWS_rain.csv', index_col=0, parse_dates=True)

In [ ]:
sun = pd.read_csv('../data/weather/Mangere_EWS_sun.csv', index_col=0, parse_dates=True)

### interpolate so that there are no missing values 

In [ ]:
temp = temp.interpolate(method='linear')

In [ ]:
rain = rain.interpolate(method='linear')

In [ ]:
sun = sun.interpolate(method='linear')

### adds the climate regressors to the data 

In [ ]:
data_with_regressors = add_regressor(data, temp, varname='temp')

In [ ]:
data_with_regressors = add_regressor(data_with_regressors, rain, varname='rain')

In [ ]:
data_with_regressors = add_regressor(data_with_regressors, sun, varname='sun')

In [ ]:
data_with_regressors.head()

In [ ]:
data_with_regressors.tail()

### prepare the data and subsets (train and test set)

In [ ]:
data_train, data_test = prepare_data(data_with_regressors, 2017)

In [ ]:
m = Prophet(mcmc_samples=300, holidays=holidays, holidays_prior_scale=0.25, changepoint_prior_scale=0.01, seasonality_mode='multiplicative', \
            yearly_seasonality=10, \
            weekly_seasonality=True, \
            daily_seasonality=False)

In [ ]:
m.add_regressor('temp', prior_scale=0.5, mode='multiplicative')
m.add_regressor('rain', prior_scale=0.5, mode='multiplicative')
m.add_regressor('sun', prior_scale=0.5, mode='multiplicative')

In [ ]:
m.fit(data_train)

In [ ]:
future = m.make_future_dataframe(periods=len(data_test), freq='1D')

In [ ]:
futures = add_regressor_to_future(future, [temp, rain, sun])

In [ ]:
forecast = m.predict(futures)

In [ ]:
f = m.plot(forecast)

In [ ]:
f = m.plot_components(forecast)

In [ ]:
verif = make_verif(forecast, data_train, data_test)

In [ ]:
verif.head()

In [ ]:
verif.loc[:,'yhat'] = verif.yhat.clip_lower(0)

In [ ]:
verif.loc[:,'yhat_lower'] = verif.yhat_lower.clip_lower(0)

In [ ]:
f =  plot_verif(verif)

In [ ]:
plt.scatter(verif.y, verif.yhat)

In [ ]:
verif.loc[:'2017',['y','yhat']].corr()

In [ ]:
sns.jointplot(x='yhat', y='y', data = verif.loc[:'2017',:])

In [ ]:
f = sns.jointplot(x='yhat', y='y', data = verif.loc['2017':])
plt.savefig('../figures/joint_plot_climate.png', dpi=200)

### Mean Absolute Error 

In [ ]:
MAE(verif.y.values, verif.yhat.values)

### plot the contribution of the different climate variables to the response variable (in percentage of the trend component, as we chose a multiplicative model)

In [ ]:
f = plot_verif_component(verif, component = 'rain')

In [ ]:
f = plot_verif_component(verif, component = 'temp')

In [ ]:
f  = plot_verif_component(verif, component = 'sun')

### plots the combined contribution of the climate extra-regressors

In [ ]:
f = plot_verif_component(verif, component = 'extra_regressors_multiplicative')

### zoom in on the post 2016 period (test set)

In [ ]:
f = plot_verif_component(verif.loc['2016-12-31':,:], component = 'extra_regressors_multiplicative')

In [ ]:
f = plot_verif_component(verif.loc['2016-12-31':,:], component = 'rain')

In [ ]:
f, ax = plt.subplots(figsize=(10,8))
verif.loc['2017-01-01':'2017-04-30',['y','yhat']].plot(lw=3, ax=ax)
ax.grid(ls=':')
f.savefig('../figures/forecasts_obs_2017-04.png', dpi=200)

### running correlations between observed and modelled / predicted values, useful to identify when things go South 

In [ ]:
corr = verif.loc[:,['y','yhat']].rolling(window=90).corr().iloc[0::2,1]

In [ ]:
corr.index = corr.index.droplevel(1)

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr.plot(ax=ax)
ax.axhline(0.7, color='0.8', alpha=0.5, zorder=-1)
ax.axhline(0.5, color='0.8', alpha=0.4, zorder=-1)